In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello World: Su Primer Circuito Cuántico

Construya un [estado de Bell](https://en.wikipedia.org/wiki/Bell_state) (dos qubits entrelazados) y ejecútelo de tres maneras:

1. **Simulación ideal** — resultados perfectos, sin cuenta necesaria
2. **Simulación con ruido** — simula un dispositivo real, sin cuenta necesaria
3. **Hardware cuántico real** — requiere una [cuenta de IBM Quantum](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)

## Construir el circuito

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## Opción 1: Simulación ideal (sin cuenta necesaria)
Usa `StatevectorSampler` — un simulador local con resultados perfectos, sin ruido.

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## Opción 2: Simulación con ruido (sin cuenta necesaria)
Usa `FakeManilaV2` — un simulador local que imita un dispositivo cuántico real de IBM, incluyendo sus características de ruido. El circuito debe ser transpilado primero (adaptado) al conjunto de gates y conectividad de qubits del dispositivo.

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Opción 3: Hardware cuántico real
Requiere una cuenta de IBM Quantum. Vea [Configurar su cuenta de IBM Quantum](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account) para más detalles.

Si aún no ha guardado sus credenciales en esta sesión de Binder, ejecute esto primero:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**Nota:** Los trabajos en hardware real pueden tomar algún tiempo dependiendo de los tiempos de cola. Si la celda aún se está ejecutando, puede verificar el estado de su trabajo y ver los resultados en [quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me).

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## ¿Qué sigue?
- **[Tutorials](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — guías paso a paso sobre algoritmos, mitigación de errores, transpilación y más
- **[Courses](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — rutas de aprendizaje estructuradas desde fundamentos cuánticos hasta computación a escala de utilidad
- **[Modo de prueba local](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — ejecute la mayoría de los notebooks sin una cuenta de IBM Quantum